# Trend-Scanning × Momentum Horizon Evaluation (Evidence-ready)

このノートブックは、`trend_scanning_reproduction.ipynb` を `%run` で読み込み、
Trend-Scanning を **状態変数（lookforward=False）** として用いながら、
モメンタムのホライズン（21/63/126/252など）が **実際に有効か** を **OOS・コスト後**で比較します。

## 出力（証跡）
- `outputs/evidence/` に図・表を自動保存
- `outputs/logs/` に設定と集計テーブルを保存

## リーク防止
- Trend-Scanning window: lookforward=False
- weights: リバランス日に決定 → **翌営業日から適用**（shift(1)）
- forward return: 評価用のみ（-fwd_days シフト）


In [ ]:
# ===== 0) Imports & Output paths =====
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# notebook実行ディレクトリ（Jupyterのcwd）を基準に outputs/ を作る
OUT_ROOT = Path("outputs")
OUT_EVI = OUT_ROOT / "evidence"
OUT_LOG = OUT_ROOT / "logs"
OUT_EVI.mkdir(parents=True, exist_ok=True)
OUT_LOG.mkdir(parents=True, exist_ok=True)

RUN_TS = pd.Timestamp.utcnow().strftime("%Y%m%d_%H%M%S")
print("RUN_TS:", RUN_TS)
print("OUT_EVI:", OUT_EVI.resolve())
print("OUT_LOG:", OUT_LOG.resolve())


In [ ]:
# ===== 1) Load existing Trend-Scanning implementation =====
# 同じディレクトリに置いてください:
# - trend_scanning_reproduction.ipynb
%run ./trend_scanning_reproduction.ipynb


In [ ]:
# ===== 2) Config =====
CFG = {
    "tickers": ["SPY","QQQ","IWM","EFA","EEM","TLT","IEF","GLD","USO"],
    "start": "2005-01-01",
    "end": None,

    # Data source: "yfinance" or "csv"
    "data_source": "yfinance",
    "csv_path": "data/prices.csv",  # data_source="csv" のとき利用（index=Date, columns=tickers）

    "rebalance": "M",  # "W" or "M"
    "fwd_days": 21,

    "mom_horizons": [21, 63, 126, 252],
    "skip_days_for_252": 21,  # 12M skip 1M

    "top_k": 3,
    "cost_bps": 10.0,

    # Trend-Scanning parameters
    "ts_span": (10, 252),
    "ts_vol_q": 0.05,
    "ts_impl": "numba",  # "numba" or "ref"

    # Gating bins for window -> horizon mapping
    "gate_bins": [30, 120],
    "gate_map": {  # short/mid/long horizon
        "short": 21,
        "mid": 63,
        "long": 252,
    },

    # Evidence file prefix
    "tag": "ts_mom_eval",
}

pd.Series(CFG)


In [ ]:
# ===== 3) Data loading =====
def load_prices_yfinance(tickers, start, end):
    import yfinance as yf
    df = yf.download(tickers, start=start, end=end, auto_adjust=True, progress=False)["Close"]
    if isinstance(df, pd.Series):
        df = df.to_frame()
    df = df.dropna(how="all").sort_index()
    df.columns = [c.upper() for c in df.columns]
    return df

def load_prices_csv(path):
    df = pd.read_csv(path, parse_dates=["Date"]).set_index("Date").sort_index()
    df.columns = [c.upper() for c in df.columns]
    return df

if CFG["data_source"] == "yfinance":
    PX = load_prices_yfinance([t.upper() for t in CFG["tickers"]], CFG["start"], CFG["end"])
elif CFG["data_source"] == "csv":
    PX = load_prices_csv(CFG["csv_path"])
else:
    raise ValueError("CFG['data_source'] must be 'yfinance' or 'csv'")

PX = PX.dropna(how="all")
print("PX:", PX.shape, PX.index.min(), PX.index.max())


In [ ]:
# ===== 4) Helpers =====
def save_df(df: pd.DataFrame, name: str):
    p = OUT_LOG / f"{CFG['tag']}_{RUN_TS}_{name}.csv"
    df.to_csv(p)
    print("saved:", p)

def save_fig(name: str):
    p = OUT_EVI / f"{CFG['tag']}_{RUN_TS}_{name}.png"
    plt.savefig(p, dpi=160, bbox_inches="tight")
    print("saved:", p)
    plt.close()

def to_rebalance_dates(px: pd.DataFrame, freq: str) -> pd.DatetimeIndex:
    if freq == "M":
        return px.resample("M").last().index
    if freq == "W":
        return px.resample("W-FRI").last().index
    raise ValueError("rebalance must be 'W' or 'M'")

def forward_return(px: pd.DataFrame, fwd_days: int) -> pd.DataFrame:
    return px.pct_change(fwd_days).shift(-fwd_days)

def normalize_weights(w: pd.DataFrame) -> pd.DataFrame:
    s = w.abs().sum(axis=1).replace(0.0, np.nan)
    return w.div(s, axis=0).fillna(0.0)

def turnover(w: pd.DataFrame) -> pd.Series:
    return w.diff().abs().sum(axis=1).fillna(0.0)

def portfolio_return(w: pd.DataFrame, fwd_ret: pd.DataFrame) -> pd.Series:
    common = w.index.intersection(fwd_ret.index)
    ww = w.loc[common]
    rr = fwd_ret.loc[common, ww.columns]
    return (ww * rr).sum(axis=1)

def apply_costs(port_ret: pd.Series, w: pd.DataFrame, cost_bps: float) -> pd.Series:
    tc = (cost_bps / 1e4) * turnover(w)
    return port_ret - tc

def monthly_rebalance_weights(w_daily: pd.DataFrame, px: pd.DataFrame, freq: str) -> pd.DataFrame:
    rb = to_rebalance_dates(px, freq).intersection(w_daily.index)
    w_rb = w_daily.loc[rb].copy()
    # 実運用：rb日で決めたウェイトは翌営業日から効かせる
    w_rb = w_rb.shift(1)
    w_hold = w_rb.reindex(px.index, method="ffill").fillna(0.0)
    return w_hold

def summarize_perf(port_ret: pd.Series, ann_factor: float) -> pd.Series:
    s = port_ret.dropna()
    if len(s) == 0:
        return pd.Series({"ann_mean": np.nan, "ann_vol": np.nan, "sharpe": np.nan, "n": 0})
    mu = s.mean() * ann_factor
    vol = s.std(ddof=0) * np.sqrt(ann_factor)
    sharpe = mu / vol if vol > 0 else np.nan
    return pd.Series({"ann_mean": mu, "ann_vol": vol, "sharpe": sharpe, "n": len(s)})

def compute_max_drawdown(cum: pd.Series) -> float:
    peak = cum.cummax()
    dd = cum / peak - 1.0
    return float(dd.min()) if len(dd) else np.nan


## 5) Trend-Scanning window（状態変数）作成

- `BENCH`（デフォルトはSPY）で `window=h*(t)` を算出
- **lookforward=False** なのでリークしません


In [ ]:
BENCH = "SPY" if "SPY" in PX.columns else PX.columns[0]

def trend_scanning_windows(close: pd.Series) -> pd.DataFrame:
    impl = CFG["ts_impl"]
    fn = trend_scanning_labels_numba if impl == "numba" else trend_scanning_labels_reference
    df = fn(
        close.dropna(),
        span=CFG["ts_span"],
        lookforward=False,
        use_log=True,
        volatility_threshold=CFG["ts_vol_q"],
    )
    return df

ts_bench = trend_scanning_windows(PX[BENCH])
win = ts_bench["window"].reindex(PX.index).ffill()

# Evidence plots
plt.figure()
win.dropna().hist(bins=40)
plt.title(f"Optimal window distribution (BENCH={BENCH})")
plt.xlabel("window (trading days)"); plt.ylabel("count")
save_fig("ts_window_hist")

plt.figure()
win.dropna().plot()
plt.title(f"Optimal window over time (BENCH={BENCH})")
plt.xlabel("date"); plt.ylabel("window (days)")
save_fig("ts_window_timeseries")

save_df(ts_bench, "ts_bench_full")


## 6) Momentum signals (CSMOM) + portfolios (LO/LS/EQW)

- Cross-sectional momentum を用いてランキングで top-k を作ります
- 252は skip 1M をオプションで実装

In [ ]:
def mom_score_crosssection(px: pd.DataFrame, h: int) -> pd.DataFrame:
    ret = px.pct_change(h)
    rk = ret.rank(axis=1, method="average", pct=True)
    return rk - 0.5

def mom_score_12m_skip(px: pd.DataFrame, h: int = 252, skip: int = 21) -> pd.DataFrame:
    px2 = px.shift(skip)
    ret = px2.pct_change(h - skip)
    rk = ret.rank(axis=1, method="average", pct=True)
    return rk - 0.5

def weights_topk_longonly(score: pd.DataFrame, top_k: int) -> pd.DataFrame:
    ranks = score.rank(axis=1, ascending=False, method="first")
    w = (ranks <= top_k).astype(float)
    w = w.div(w.sum(axis=1).replace(0.0, np.nan), axis=0).fillna(0.0)
    return w

def weights_topk_longshort(score: pd.DataFrame, k: int) -> pd.DataFrame:
    ranks_hi = score.rank(axis=1, ascending=False, method="first")
    ranks_lo = score.rank(axis=1, ascending=True, method="first")
    long = (ranks_hi <= k).astype(float)
    short = (ranks_lo <= k).astype(float) * (-1.0)
    w = long + short
    w = normalize_weights(w)
    return w

def weights_eqw(px: pd.DataFrame) -> pd.DataFrame:
    w = pd.DataFrame(1.0, index=px.index, columns=px.columns)
    return w.div(w.sum(axis=1), axis=0)

# Scores by horizon
MOM_CS = {}
for h in CFG["mom_horizons"]:
    if h == 252 and CFG["skip_days_for_252"] is not None:
        MOM_CS[h] = mom_score_12m_skip(PX, h=252, skip=CFG["skip_days_for_252"])
    else:
        MOM_CS[h] = mom_score_crosssection(PX, h)

FWD = forward_return(PX, CFG["fwd_days"])
ann_factor = 12.0 if CFG["rebalance"] == "M" else 52.0


In [ ]:
def compute_ic(score: pd.DataFrame, fwd_ret: pd.DataFrame, freq: str) -> pd.Series:
    rb = to_rebalance_dates(score, freq).intersection(score.index).intersection(fwd_ret.index)
    s = score.loc[rb]
    r = fwd_ret.loc[rb, s.columns]
    return s.corrwith(r, axis=1)

def eval_strategy(name: str, w_daily: pd.DataFrame) -> dict:
    w = monthly_rebalance_weights(w_daily, PX, CFG["rebalance"])
    pr = portfolio_return(w, FWD)
    pr_c = apply_costs(pr, w, CFG["cost_bps"])
    cum = (1.0 + pr_c.fillna(0.0)).cumprod()
    out = summarize_perf(pr_c, ann_factor).to_dict()
    out["mdd"] = compute_max_drawdown(cum)
    out["avg_turnover"] = float(turnover(w).mean())
    return out, pr_c, w

# EQW baseline
res_rows = []
ret_map = {}
w_map = {}

out, pr, w = eval_strategy("EQW", weights_eqw(PX))
res_rows.append({"name":"EQW","h":"NA","style":"EQW", **out, "IC_mean": np.nan})
ret_map["EQW"] = pr
w_map["EQW"] = w

# Horizon strategies
for h, score in MOM_CS.items():
    ic = compute_ic(score, FWD, CFG["rebalance"]).mean()

    out_lo, pr_lo, w_lo = eval_strategy(f"CSMOM_{h}_LO", weights_topk_longonly(score, CFG["top_k"]))
    res_rows.append({"name":f"CSMOM_{h}", "h":h, "style":"LongOnly", **out_lo, "IC_mean": float(ic)})
    ret_map[f"CSMOM{h}_LO"] = pr_lo
    w_map[f"CSMOM{h}_LO"] = w_lo

    out_ls, pr_ls, w_ls = eval_strategy(f"CSMOM_{h}_LS", weights_topk_longshort(score, CFG["top_k"]))
    res_rows.append({"name":f"CSMOM_{h}", "h":h, "style":"LongShort", **out_ls, "IC_mean": float(ic)})
    ret_map[f"CSMOM{h}_LS"] = pr_ls
    w_map[f"CSMOM{h}_LS"] = w_ls

summary = pd.DataFrame(res_rows)
summary = summary.sort_values(["style","sharpe"], ascending=[True, False]).reset_index(drop=True)
summary


In [ ]:
# Save summary & config
save_df(summary, "summary_metrics")
pd.Series(CFG).to_csv(OUT_LOG / f"{CFG['tag']}_{RUN_TS}_config.csv")
print("saved config")

# Cum plot: pick a subset to avoid clutter
plt.figure()
for k in ["EQW"] + [f"CSMOM{h}_LO" for h in CFG["mom_horizons"]]:
    cs = (1.0 + ret_map[k].fillna(0.0)).cumprod()
    cs.plot(label=k)
plt.title(f"Long-only cumulative (top-{CFG['top_k']}, cost {CFG['cost_bps']}bps)")
plt.xlabel("date"); plt.ylabel("cumulative")
plt.legend()
save_fig("cum_longonly_all_h")

plt.figure()
for k in ["EQW"] + [f"CSMOM{h}_LS" for h in CFG["mom_horizons"]]:
    cs = (1.0 + ret_map[k].fillna(0.0)).cumprod()
    cs.plot(label=k)
plt.title(f"Long-short cumulative (k={CFG['top_k']}, cost {CFG['cost_bps']}bps)")
plt.xlabel("date"); plt.ylabel("cumulative")
plt.legend()
save_fig("cum_longshort_all_h")


## 7) Trend-Scanning gating: window → horizon

window を short/mid/long の3状態に粗く分け、
各状態で異なるモメンタムホライズンのスコアを採用します。

In [ ]:
def gate_horizon_from_window(window_series: pd.Series, gate_bins, h_short, h_mid, h_long) -> pd.Series:
    b0, b1 = gate_bins
    w = window_series.copy()
    out = pd.Series(index=w.index, dtype=float)
    out[w <= b0] = h_short
    out[(w > b0) & (w <= b1)] = h_mid
    out[w > b1] = h_long
    return out.astype(int)

gate_h = gate_horizon_from_window(
    win,
    CFG["gate_bins"],
    CFG["gate_map"]["short"],
    CFG["gate_map"]["mid"],
    CFG["gate_map"]["long"],
)

# Save regime counts
reg = gate_h.value_counts().sort_index().to_frame("count")
save_df(reg, "gate_h_counts")
reg


In [ ]:
def gated_score(score_by_h: dict, gate_h: pd.Series) -> pd.DataFrame:
    # score_by_h: {h: score_df}
    any_df = next(iter(score_by_h.values()))
    idx = any_df.index.intersection(gate_h.index)
    cols = any_df.columns
    out = pd.DataFrame(index=idx, columns=cols, dtype=float)

    for h, sc in score_by_h.items():
        m = (gate_h.loc[idx] == int(h)).to_numpy()
        if m.any():
            out.loc[idx[m], :] = sc.loc[idx[m], :].values
    return out

g_score = gated_score(MOM_CS, gate_h)

out_glo, pr_glo, w_glo = eval_strategy("GATED_LO", weights_topk_longonly(g_score, CFG["top_k"]))
out_gls, pr_gls, w_gls = eval_strategy("GATED_LS", weights_topk_longshort(g_score, CFG["top_k"]))

ret_map["GATED_LO"] = pr_glo
ret_map["GATED_LS"] = pr_gls
w_map["GATED_LO"] = w_glo
w_map["GATED_LS"] = w_gls

g_summary = pd.DataFrame([
    {"name":"GATED", "h":"gated", "style":"LongOnly", **out_glo, "IC_mean": float(compute_ic(g_score, FWD, CFG["rebalance"]).mean())},
    {"name":"GATED", "h":"gated", "style":"LongShort", **out_gls, "IC_mean": float(compute_ic(g_score, FWD, CFG["rebalance"]).mean())},
])
save_df(g_summary, "gated_metrics")
g_summary


In [ ]:
# Plot gated vs best fixed (by sharpe) + EQW
def best_key(style: str, suffix: str):
    ss = summary[summary["style"]==style].copy()
    ss = ss.sort_values("sharpe", ascending=False)
    if len(ss)==0: return None
    h = int(ss.iloc[0]["h"])
    return f"CSMOM{h}_{suffix}"

best_lo = best_key("LongOnly", "LO")
best_ls = best_key("LongShort", "LS")

plt.figure()
for k in ["EQW", "GATED_LO"] + ([best_lo] if best_lo else []):
    cs = (1.0 + ret_map[k].fillna(0.0)).cumprod()
    cs.plot(label=k)
plt.title("Gated vs Best Fixed (Long-only)")
plt.xlabel("date"); plt.ylabel("cumulative")
plt.legend()
save_fig("cum_gated_vs_best_lo")

plt.figure()
for k in ["EQW", "GATED_LS"] + ([best_ls] if best_ls else []):
    cs = (1.0 + ret_map[k].fillna(0.0)).cumprod()
    cs.plot(label=k)
plt.title("Gated vs Best Fixed (Long-short)")
plt.xlabel("date"); plt.ylabel("cumulative")
plt.legend()
save_fig("cum_gated_vs_best_ls")


## 8) Critical checks (multiple testing / turnover / leak)

このセルは“結果が良くても信用して良いか”を点検するための最低限のチェックです。

In [ ]:
# Turnover compare
turn_df = pd.DataFrame({k: turnover(w) for k, w in w_map.items()})
save_df(turn_df.describe().T, "turnover_desc")

plt.figure()
turn_df["GATED_LO"].dropna().hist(bins=40)
plt.title("Turnover distribution: GATED_LO")
plt.xlabel("L1 turnover"); plt.ylabel("count")
save_fig("turnover_hist_gated_lo")

# Leak sanity: weights are shifted (should start later than first rebalance)
print("first nonzero date EQW:", w_map["EQW"].abs().sum(1).ne(0).idxmax())
print("first nonzero date GATED_LO:", w_map["GATED_LO"].abs().sum(1).ne(0).idxmax())


In [ ]:
# Save returns for offline analysis
rets = pd.DataFrame(ret_map).sort_index()
save_df(rets, "strategy_returns_cost_adj")

print("Done. Evidence saved under outputs/.")
